done


In [35]:
import json
# import nltk
import pickle
from collections import Counter
import  pickle
import datetime
import copy
import json
# import numpy as np
# import matplotlib.pyplot as plt
# from matplotlib.collections import PatchCollection
# from matplotlib.patches import Polygon
# from skimage.draw import polygon
import math
import json
import argparse
import torch
import torch.nn as nn
import numpy as np
import os
import pickle
from torch.autograd import Variable 
from torchvision import transforms
from torch.nn.utils.rnn import pack_padded_sequence
import json
import torch
import torch.nn as nn
import numpy as np
import os
import glob
import pickle
from torch.autograd import Variable 
from torchvision import transforms, datasets
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torchvision.models as models
from torch.nn.utils.rnn import pack_padded_sequence
from torch.autograd import Variable
import torch.nn.functional as F
from torch.nn import init
import torch
import torchvision.transforms as transforms
import torch.utils.data as data
import os
import string
import numpy as np
#import nltk
from PIL import Image

## Creating a dictionary split for training, validation, and testing
Input data:
1. train2014 folder with images
2. val2014 folder with images
3. annotations folder with caption_train2014.json, caption_val2014.json

json file is just a dictionary of data regarding images: url, name, id, caption etc.


In [36]:
# --------------------------------------imports-----------------------------------------
# Convert the json annotation files into dictionary
val = json.load(open('C:/Users/aniketdl/Downloads/data/captions/annotations/captions_val2014.json', 'r'))
train = json.load(open('C:/Users/aniketdl/Downloads/data/captions/annotations/captions_train2014.json', 'r'))

# Merge together
imgs = val['images'] + train['images']  # list of dictionaries of image data
annots = val['annotations'] + train['annotations']  # list of dictionaries of caption data

# Creates a split dictionary dataset of images with key as type and value as list of images

dataset = dict()
dataset['val'] = imgs[:5000]
dataset['train'] = imgs[5000: 10000]
dataset['test'] = imgs[10000: 15000]
# we made dictionary with images, 5000 each
# dataset has key as split and value as list of image data

# idtoann is a dictionary with key as image_id and value as corresponding annotation (dictionary)
idtoann = dict()
for a in annots:
    imgid = a['image_id']
    if imgid not in idtoann:
        idtoann[imgid] = list()
    idtoann[imgid].append(a)
# itoann[image_id] = [{caption, id, image_id}, {caption, id, image_id}, {caption, id, image_id}.........]

# split_data is a dictionary with splits as the key
# in each value we have, ['images'] and ['annotations'] list with each list in order of image_id
split_data = dict()
splits = ['val', 'test', 'train']

for split in splits:

    split_data[split] = {'images': [], 'annotations': []}
    for img in dataset[split]:
        img_id = img['id']
        anns = idtoann[img_id]

        split_data[split]['images'].append(img)
        split_data[split]['annotations'].extend(anns)

#    json.dump(split_data[split], open('./annotations/split_' + split + '.json', 'w'))
print('done')

done


## BUILDING VOCABULARY
Use the preprocessed vocab.pkl file as an alternative, if the 'nltk' import fails

In [37]:
# nltk.download('punkt')

# VOCABULARY object contains word2idx and idx2word to get values for word and idx including of
# <START>, <END> and <UNK>
class Vocabulary():
    def __init__(self):
        self.word2idx = dict()
        self.idx2word = dict()
        self.idx = 0  # initial value for indexing

    # the function add_word
    def add_word(self, word):
        if word not in self.word2idx:
            self.word2idx[word] = self.idx
            self.idx2word[self.idx] = word
            self.idx += 1

    def __call__(self, word):
        if not word in self.word2idx:
            return self.word2idx['<unk>']
        return self.word2idx[word]

    def __len__(self):
        return len(self.word2idx)

# def build_vocab(split, threshold):
#     """Build a simple vocabulary wrapper."""
#     coco = COCO(split)
#     counter = Counter()
#     ids = coco.anns.keys()
#     for i, id in enumerate(ids):
#         caption = str(coco.anns[id]['caption'])
#         tokens = nltk.tokenize.word_tokenize(caption.lower())
#         counter.update(tokens)

#         if i % 1000 == 0:
#             print("[{} {}] Tokenized the captions.".format(i, len(ids)))

#     # If the word frequency is less than 'threshold', then the word is discarded.
#     words = [word for word, cnt in counter.items() if cnt >= threshold]

#     # Creates a vocab wrapper and add some special tokens.
#     vocab = Vocabulary()
#     vocab.add_word('<pad>')
#     vocab.add_word('<start>')
#     vocab.add_word('<end>')
#     vocab.add_word('<unk>')

#     # Adds the words to the vocabulary.
#     for i, word in enumerate(words):
#         vocab.add_word(word)
#     return vocab


# vocab = build_vocab(split=split_data,
#                         threshold=5)
# this is a Vocabulary object with word2idx and idx2word including
# start, end, unk and pad tokens
# here pad = 0, start = 1, end = 2, unk = 3
print('done')

done


In [38]:
with open('C:/Users/aniketdl/Downloads/data/vocab.pkl', 'rb') as f:
        vocab = pickle.load(f)

## COCO object creation for future use

In [39]:
# COCO object is created to be used in future for vocabulary creation
class COCO:
    def __init__(self, split_data=None):
        """
        :param annotations (dict): annotation dictionary with train, test and val data
        :param im   age_folder (str): location to the folder that hosts images.
        :return:
        """
        self.dataset = {}  # this is basically split_data['train'] having images and annotation lists
        self.anns = {}  # a dictionary with key as id(caption) and value as annotation dict
        self.imgToAnns = {} # a dictionary with key as image_id and value as list of annotation dicts
        self.imgs = {}  # a dictionary with key as image_id and value as image dict

        if split_data != None:
            print('loading annotations into memory...')
            time_t = datetime.datetime.utcnow()
            dataset = split_data['train']  # dataset is dictionary with "images" and "annotations" keys
            print(datetime.datetime.utcnow() - time_t)
            self.dataset = dataset
            self.createIndex()

    def createIndex(self):
        print('creating index...')
        imgToAnns = {ann['image_id']: [] for ann in self.dataset['annotations']}
        anns = {ann['id']:       [] for ann in self.dataset['annotations']}
        for ann in self.dataset['annotations']:
            imgToAnns[ann['image_id']] += [ann]
            anns[ann['id']] = ann

        imgs = {im['id']: {} for im in self.dataset['images']}
        for img in self.dataset['images']:
            imgs[img['id']] = img
        print('index created!')

        # create class members
        self.anns = anns
        self.imgToAnns = imgToAnns
        self.imgs = imgs

    # for given imgIds, it returns list of corresponding annotation ids (caption id)
    def getAnnIds(self, imgIds=[], areaRng=[], iscrowd=None):
        """
        Get ann ids that satisfy given filter conditions. default skips that filter
        :param imgIds  (int array)     : get anns for given imgs
               areaRng (float array)   : get anns for given area range (e.g. [0 inf])
        :return: ids (int array)       : integer array of ann ids
        """
        imgIds = imgIds if type(imgIds) == list else [imgIds]

        if len(imgIds) == len(areaRng) == 0:
            anns = self.dataset['annotations']
        else:
            if not len(imgIds) == 0:
                anns = sum([self.imgToAnns[imgId] for imgId in imgIds if imgId in self.imgToAnns],[])
            else:
                anns = self.dataset['annotations']
            anns = anns if len(areaRng) == 0 else [ann for ann in anns if ann['area'] > areaRng[0] and ann['area'] < areaRng[1]]
        if self.dataset['type'] == 'instances':
            if not iscrowd == None:
                ids = [ann['id'] for ann in anns if ann['iscrowd'] == iscrowd]
            else:
                ids = [ann['id'] for ann in anns]
        else:
            ids = [ann['id'] for ann in anns]
        return ids

    # returns imgids which satisfy the filter condition
    # kind of a filter on image ids
    def getImgIds(self, imgIds=[]):
        '''
        Get img ids that satisfy given filter conditions.
        :param imgIds (int array) : get imgs for given ids
        :return: ids (int array)  : integer array of img ids
        '''
        imgIds = imgIds if type(imgIds) == list else [imgIds]
        if len(imgIds) == 0:
            ids = self.imgs.keys()
        else:
            ids = set(imgIds)
        return list(ids)


    # returns annotation for a given list of annot ids
    # can be used with getAnnIds as # dataset.loadAnns(dataset.getAnnIds([410328]))
    # to return all the annotations for a given image_id
    def loadAnns(self, ids=[]):
        """
        Load anns with the specified ids.
        :param ids (int array)       : integer ids specifying anns
        :return: anns (object array) : loaded ann objects
        """
        if type(ids) == list:
            return [self.anns[id] for id in ids]
        elif type(ids) == int:
            return [self.anns[ids]]

    # returns the image data for a given image_id
    def loadImgs(self, ids=[]):
        """
        Load anns with the specified ids.
        :param ids (int array)       : integer ids specifying img
        :return: imgs (object array) : loaded img objects
        """
        if type(ids) == list:
            return [self.imgs[id] for id in ids]
        elif type(ids) == int:
            return [self.imgs[ids]]

    # #
    # def showAnns(self, anns):
    #     """
    #     Display the specified annotations.
    #     :param anns (array of object): annotations to display
    #     :return: None
    #     """
    #     if len(anns) == 0:
    #         return 0
    #     ax = plt.gca()
    #     polygons = []
    #     color = []
    #     for ann in anns:
    #         c = np.random.random((1, 3)).tolist()[0]
    #         if type(ann['segmentation']) == list:
    #             # polygon
    #             for seg in ann['segmentation']:
    #                 poly = np.array(seg).reshape((len(seg)/2, 2))
    #                 polygons.append(Polygon(poly, True,alpha=0.4))
    #                 color.append(c)
    #         else:
    #             # mask
    #             mask = COCO.decodeMask(ann['segmentation'])
    #             img = np.ones( (mask.shape[0], mask.shape[1], 3) )
    #             if ann['iscrowd'] == 1:
    #                 color_mask = np.array([2.0,166.0,101.0])/255
    #             if ann['iscrowd'] == 0:
    #                 color_mask = np.random.random((1, 3)).tolist()[0]
    #             for i in range(3):
    #                 img[:,:,i] = color_mask[i]
    #             ax.imshow(np.dstack( (img, mask*0.5) ))
    #     p = PatchCollection(polygons, facecolors=color, edgecolors=(0,0,0,1), linewidths=3, alpha=0.4)
    #     ax.add_collection(p)
    #
    #     if self.dataset['type'] == 'captions':
    #         for ann in anns:
    #             print(ann['caption'])

    #
    def loadRes(self, resFile):
        """
        Load result file and return a result api object.
        :param   resFile (str)     : file name of result file
        :return: res (obj)         : result api object
        """
        res = COCO()
        res.dataset['images'] = [img for img in self.dataset['images']]
        res.dataset['info'] = copy.deepcopy(self.dataset['info'])
        res.dataset['type'] = copy.deepcopy(self.dataset['type'])
        res.dataset['licenses'] = copy.deepcopy(self.dataset['licenses'])

        print('Loading and preparing results...     ')
        time_t = datetime.datetime.utcnow()
        anns    = json.load(open(resFile))
        assert type(anns) == list, 'results in not an array of objects'
        annsImgIds = [ann['image_id'] for ann in anns]
        assert set(annsImgIds) == (set(annsImgIds) & set(self.getImgIds())), \
               'Results do not correspond to current coco set'
        if 'caption' in anns[0]:
            imgIds = set([img['id'] for img in res.dataset['images']]) & set([ann['image_id'] for ann in anns])
            res.dataset['images'] = [img for img in res.dataset['images'] if img['id'] in imgIds]
            for id, ann in enumerate(anns):
                ann['id'] = id
        elif 'bbox' in anns[0] and not anns[0]['bbox'] == []:
            res.dataset['categories'] = copy.deepcopy(self.dataset['categories'])
            for id, ann in enumerate(anns):
                bb = ann['bbox']
                x1, x2, y1, y2 = [bb[0], bb[0]+bb[2], bb[1], bb[1]+bb[3]]
                ann['segmentation'] = [[x1, y1, x1, y2, x2, y2, x2, y1]]
                ann['area'] = bb[2]*bb[3]
                ann['id'] = id
                ann['iscrowd'] = 0
        elif 'segmentation' in anns[0]:
            res.dataset['categories'] = copy.deepcopy(self.dataset['categories'])
            for id, ann in enumerate(anns):
                ann['area']=sum(ann['segmentation']['counts'][2:-1:2])
                ann['bbox'] = []
                ann['id'] = id
                ann['iscrowd'] = 0
        print('DONE (t={}s)'.format((datetime.datetime.utcnow() - time_t).total_seconds()))

        res.dataset['annotations'] = anns
        res.createIndex()
        return res
    #
    #
    # @staticmethod
    # def decodeMask(R):
    #     """
    #     Decode binary mask M encoded via run-length encoding.
    #     :param   R (object RLE)    : run-length encoding of binary mask
    #     :return: M (bool 2D array) : decoded binary mask
    #     """
    #     N = len(R['counts'])
    #     M = np.zeros( (R['size'][0]*R['size'][1], ))
    #     n = 0
    #     val = 1
    #     for pos in range(N):
    #         val = not val
    #         for c in range(R['counts'][pos]):
    #             R['counts'][pos]
    #             M[n] = val
    #             n += 1
    #     return M.reshape((R['size']), order='F')
    #
    # @staticmethod
    # def encodeMask(M):
    #     """
    #     Encode binary mask M using run-length encoding.
    #     :param   M (bool 2D array)  : binary mask to encode
    #     :return: R (object RLE)     : run-length encoding of binary mask
    #     """
    #     [h, w] = M.shape
    #     M = M.flatten(order='F')
    #     N = len(M)
    #     counts_list = []
    #     pos = 0
    #     # counts
    #     counts_list.append(1)
    #     diffs = np.logical_xor(M[0:N-1], M[1:N])
    #     for diff in diffs:
    #         if diff:
    #             pos +=1
    #             counts_list.append(1)
    #         else:
    #             counts_list[pos] += 1
    #     # if array starts from 1. start with 0 counts for 0
    #     if M[0] == 1:
    #         counts_list = [0] + counts_list
    #     return {'size':      [h, w],
    #            'counts':    counts_list ,
    #            }
    #
    # @staticmethod
    # def segToMask( S, h, w ):
    #      """
    #      Convert polygon segmentation to binary mask.
    #      :param   S (float array)   : polygon segmentation mask
    #      :param   h (int)           : target mask height
    #      :param   w (int)           : target mask width
    #      :return: M (bool 2D array) : binary mask
    #      """
    #      M = np.zeros((h,w), dtype=np.bool)
    #      for s in S:
    #          N = len(s)
    #          rr, cc = polygon(np.array(s[1:N:2]), np.array(s[0:N:2])) # (y, x)
    #          M[rr, cc] = 1
    #      return M

In [40]:
max_len = 40

class CocoDataset(data.Dataset):
    def __init__(self, root, split_data, vocab, transform=None):
        """
            root: image directory. data/
            vocab: vocabulary wrapper.
            transform: image transformer.
        """
        self.root = root
        self.coco = COCO(split_data)  # creates a COCO object with various properties and methods
        self.ids = list(self.coco.anns.keys())  # list of annotation ids
        self.vocab = vocab  # vocabulary for training
        self.transform = transform  # transforms if given

    # item returned in each iteration
    def __getitem__(self, index):
        """Returns one data pair ( image, caption, image_id )."""
        coco = self.coco
        vocab = self.vocab
        ann_id = self.ids[index]
        caption = coco.anns[ann_id]['caption']  # returns caption string
        img_id = coco.anns[ann_id]['image_id']  # returns image id
        filename = coco.loadImgs(img_id)[0]['file_name']  # returns name of image

        if 'val' in filename.lower():
            path = 'val2014/' + filename
        else:
            path = 'train2014/' + filename

        image = Image.open(os.path.join(self.root, path)).convert('RGB')
        if self.transform is not None:
            image = self.transform(image)

        # Convert caption (string) to word ids.
        txt1 = str(caption).lower()
        mytable = txt1.maketrans(string.punctuation, str(" "*len(string.punctuation)))
        tokens = txt1.translate(mytable).strip().split()
        caption = []
        caption.append(vocab('<start>'))
        caption.extend([vocab(token) for token in tokens])
        caption.append(vocab('<end>'))
        for i in range(max_len - len(caption)):
            caption.append(vocab('<pad>'))
        target = torch.Tensor(caption)
        return image, target, img_id, filename

    def __len__(self):
        return len(self.ids)


def collate_fn(data):
    """Creates mini-batch tensors from the list of tuples (image, caption).

    We should build custom collate_fn rather than using default collate_fn,
    because merging caption (including padding) is not supported in default.
    Args:
        data: list of tuple (image, caption).
            - image: torch tensor of shape (3, 256, 256).
            - caption: torch tensor of shape (?); variable length.
    Returns:
        images: torch tensor of shape (batch_size, 3, 256, 256).
        targets: torch tensor of shape (batch_size, padded_length).
        lengths: list; valid length for each padded caption.
        img_ids: image ids in COCO dataset, for evaluation purpose
        filenames: image filenames in COCO dataset, for evaluation purpose
    """

    # Sort a data list by caption length (descending order).
    data.sort(key=lambda x: len(x[1]), reverse=True)
    images, captions, img_ids, filenames = zip(*data)  # unzip

    # Merge images (from tuple of 3D tensor to 4D tensor).
    images = torch.stack(images, 0)
    img_ids = list(img_ids)
    filenames = list(filenames)

    # Merge captions (from tuple of 1D tensor to 2D tensor).
    lengths = [len(cap) for cap in captions]
    targets = torch.zeros(len(captions), max(lengths)).long()
    for i, cap in enumerate(captions):
        end = lengths[i]
        targets[i, :end] = cap[:end]
    return images, targets, lengths, img_ids, filenames


#get_loader returns the data loader for training
def get_loader(root, split_data, vocab, transform, batch_size, shuffle, num_workers):
    """Returns torch.utils.data.DataLoader for custom coco dataset."""
    # COCO caption dataset
    coco = CocoDataset(root=root,
                       split_data=split_data,
                       vocab=vocab,
                       transform=transform)
    # return dataset with items (image object, numericalized caption with start and end token, image_id, filename)
    # images: tensor of shape (batch_size, 3, 224, 224).
    # captions: tensor of shape (batch_size, padded_length).
    # lengths: list indicating valid length for each caption. length is (batch_size).
    data_loader = torch.utils.data.DataLoader(dataset=coco,
                                              batch_size=batch_size,
                                              shuffle=shuffle,
                                              num_workers=num_workers,
                                              collate_fn=collate_fn)
    return data_loader

## ADAPTIVE ATTENTION MODEL

In [41]:
#  ========================================Knowing When to Look========================================
# AttentiveCNN created with embed size and hidden size
# it takes input images and outputs the feature vector V and v_g
# V: image feature vector (where) and v_g: data to be fed to Decoder
class AttentiveCNN(nn.Module):
    def __init__(self, embed_size, hidden_size):  # (256, 512)
        super(AttentiveCNN, self).__init__()

        # ResNet-152 backend
        resnet = models.resnet152(pretrained=True)
        modules = list(resnet.children())[:-2]  # delete the last fc layer and avg pool.
        resnet_conv = nn.Sequential(*modules)  # last conv feature

        self.resnet_conv = resnet_conv    # resnet model with last 2 layers removed
        self.avgpool = nn.AvgPool2d(7)    # output layer with features of size 7x7
        self.affine_a = nn.Linear(2048, hidden_size)  # 512|v_i = W_a * A   # affine transformation ensures that points
        self.affine_b = nn.Linear(2048, embed_size)  # 256|v_g = W_b * a^g  # lie on same ratio/dist after transform

        # Dropout before affine transformation
        self.dropout = nn.Dropout(0.5)
        self.init_weights()

    def init_weights(self):
        """Initialize the weights."""
        init.kaiming_uniform(self.affine_a.weight, mode='fan_in')  # He weight initialization
        init.kaiming_uniform(self.affine_b.weight, mode='fan_in')
        self.affine_a.bias.data.fill_(0)  # initializing biases
        self.affine_b.bias.data.fill_(0)

    def forward(self, images):
        """
        Input: images
        Output: V=[v_1, ..., v_n], v_g
        """

        # Last conv layer feature map
        A = self.resnet_conv(images)  # resnet conv output

        # a^g, average pooling feature map
        a_g = self.avgpool(A)  # transforms output to 7x7
        a_g = a_g.view(a_g.size(0), -1)  # reshapes it to linear

        # V = [ v_1, v_2, ..., v_49 ]
        V = A.view(A.size(0), A.size(1), -1).transpose(1, 2)
        V = F.relu(self.affine_a(self.dropout(V)))

        v_g = F.relu(self.affine_b(self.dropout(a_g)))

        return V, v_g
        # returns feature vector V and v_g


# Attention Block for C_hat calculation
class Atten(nn.Module):
    def __init__(self, hidden_size):
        super(Atten, self).__init__()

        self.affine_v = nn.Linear(hidden_size, 49, bias=False)  # W_v
        self.affine_g = nn.Linear(hidden_size, 49, bias=False)  # W_g
        self.affine_s = nn.Linear(hidden_size, 49, bias=False)  # W_s
        self.affine_h = nn.Linear(49, 1, bias=False)  # w_h

        self.dropout = nn.Dropout(0.5)
        self.init_weights()

    def init_weights(self):
        """Initialize the weights."""
        init.xavier_uniform(self.affine_v.weight)
        init.xavier_uniform(self.affine_g.weight)
        init.xavier_uniform(self.affine_h.weight)
        init.xavier_uniform(self.affine_s.weight)

    def forward(self, V, h_t, s_t):
        """
        Input: V=[v_1, v_2, ... v_k], h_t, s_t from LSTM
        Output: c_hat_t, attention feature map
        """

        # W_v * V + W_g * h_t * 1^T
        content_v = self.affine_v(self.dropout(V)).unsqueeze(1) \
                    + self.affine_g(self.dropout(h_t)).unsqueeze(2)

        # z_t = W_h * tanh( content_v )
        z_t = self.affine_h(self.dropout(F.tanh( content_v))).squeeze(3)
        alpha_t = F.softmax(z_t.view(-1, z_t.size(2))).view(z_t.size(0), z_t.size(1), -1)

        # Construct c_t: B x seq x hidden_size
        c_t = torch.bmm(alpha_t, V).squeeze(2)

        # W_s * s_t + W_g * h_t
        content_s = self.affine_s( self.dropout(s_t)) + self.affine_g(self.dropout(h_t))
        # w_t * tanh( content_s )
        z_t_extended = self.affine_h(self.dropout(F.tanh(content_s)))

        # Attention score between sentinel and image content
        extended = torch.cat((z_t, z_t_extended), dim=2)
        alpha_hat_t = F.softmax(extended.view(-1, extended.size(2))).view(extended.size(0), extended.size(1),-1)
        beta_t = alpha_hat_t[:, :, -1]

        # c_hat_t = beta * s_t + ( 1 - beta ) * c_t
        beta_t = beta_t.unsqueeze(2)
        c_hat_t = beta_t * s_t + (1 - beta_t) * c_t

        return c_hat_t, alpha_t, beta_t


# Sentinel BLock
class Sentinel(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(Sentinel, self).__init__()

        self.affine_x = nn.Linear(input_size, hidden_size, bias=False)
        self.affine_h = nn.Linear(hidden_size, hidden_size, bias=False)

        # Dropout applied before affine transformation
        self.dropout = nn.Dropout(0.5)

        self.init_weights()

    def init_weights(self):
        init.xavier_uniform(self.affine_x.weight)
        init.xavier_uniform(self.affine_h.weight)

    def forward(self, x_t, h_t_1, cell_t):

        # g_t = sigmoid( W_x * x_t + W_h * h_(t-1) )
        gate_t = self.affine_x(self.dropout(x_t)) + self.affine_h(self.dropout(h_t_1))
        gate_t = F.sigmoid(gate_t)

        # Sentinel embedding
        s_t = gate_t * F.tanh(cell_t)

        return s_t


# Adaptive Attention Block: C_t, Spatial Attention Weights, Sentinel embedding
class AdaptiveBlock( nn.Module ):

    def __init__( self, embed_size, hidden_size, vocab_size ):
        super( AdaptiveBlock, self ).__init__()

        # Sentinel block
        self.sentinel = Sentinel( embed_size * 2, hidden_size )

        # Image Spatial Attention Block
        self.atten = Atten( hidden_size )

        # Final Caption generator
        self.mlp = nn.Linear( hidden_size, vocab_size )

        # Dropout layer inside Affine Transformation
        self.dropout = nn.Dropout( 0.5 )

        self.hidden_size = hidden_size
        self.init_weights()

    def init_weights( self ):
        '''
        Initialize final classifier weights
        '''
        init.kaiming_normal( self.mlp.weight, mode='fan_in' )
        self.mlp.bias.data.fill_( 0 )


    def forward( self, x, hiddens, cells, V ):

        # hidden for sentinel should be h0-ht-1
        h0 = self.init_hidden( x.size(0) )[0].transpose( 0,1 )

        # h_(t-1): B x seq x hidden_size ( 0 - t-1 )
        if hiddens.size( 1 ) > 1:
            hiddens_t_1 = torch.cat( ( h0, hiddens[ :, :-1, : ] ), dim=1 )
        else:
            hiddens_t_1 = h0

        # Get Sentinel embedding, it's calculated blockly
        sentinel = self.sentinel( x, hiddens_t_1, cells )

        # Get C_t, Spatial attention, sentinel score
        c_hat, atten_weights, beta = self.atten( V, hiddens, sentinel )

        # Final score along vocabulary
        scores = self.mlp( self.dropout( c_hat + hiddens ) )

        return scores, atten_weights, beta

    def init_hidden( self, bsz ):
        '''
        Hidden_0 & Cell_0 initialization
        '''
        weight = next( self.parameters() ).data

        if torch.cuda.is_available():
            return ( Variable( weight.new( 1 , bsz, self.hidden_size ).zero_().cuda() ),
                    Variable( weight.new( 1,  bsz, self.hidden_size ).zero_().cuda() ) )
        else:
            return ( Variable( weight.new( 1 , bsz, self.hidden_size ).zero_() ),
                    Variable( weight.new( 1,  bsz, self.hidden_size ).zero_() ) )


# Caption Decoder
class Decoder( nn.Module ):
    def __init__( self, embed_size, vocab_size, hidden_size ):
        super( Decoder, self ).__init__()

        # word embedding
        self.embed = nn.Embedding( vocab_size, embed_size )

        # LSTM decoder: input = [ w_t; v_g ] => 2 x word_embed_size;
        self.LSTM = nn.LSTM( embed_size * 2, hidden_size, 1, batch_first=True )

        # Save hidden_size for hidden and cell variable
        self.hidden_size = hidden_size

        # Adaptive Attention Block: Sentinel + C_hat + Final scores for caption sampling
        self.adaptive = AdaptiveBlock( embed_size, hidden_size, vocab_size )

    def forward( self, V, v_g , captions, states=None ):

        # Word Embedding
        embeddings = self.embed( captions )

        # x_t = [w_t;v_g]
        x = torch.cat( ( embeddings, v_g.unsqueeze( 1 ).expand_as( embeddings ) ), dim=2 )

        # Hiddens: Batch x seq_len x hidden_size
        # Cells: seq_len x Batch x hidden_size, default setup by Pytorch
        if torch.cuda.is_available():
            hiddens = Variable( torch.zeros( x.size(0), x.size(1), self.hidden_size ).cuda() )
            cells = Variable( torch.zeros( x.size(1), x.size(0), self.hidden_size ).cuda() )
        else:
            hiddens = Variable( torch.zeros( x.size(0), x.size(1), self.hidden_size ) )
            cells = Variable( torch.zeros( x.size(1), x.size(0), self.hidden_size ) )

        # Recurrent Block
        # Retrieve hidden & cell for Sentinel simulation
        for time_step in range( x.size( 1 ) ):

            # Feed in x_t one at a time
            x_t = x[ :, time_step, : ]
            x_t = x_t.unsqueeze( 1 )

            h_t, states = self.LSTM( x_t, states )

            # Save hidden and cell
            hiddens[ :, time_step, : ] = h_t  # Batch_first
            cells[ time_step, :, : ] = states[ 1 ]

        # cell: Batch x seq_len x hidden_size
        cells = cells.transpose( 0, 1 )

        # Data parallelism for adaptive attention block
        if torch.cuda.device_count() > 1:
            device_ids = range( torch.cuda.device_count() )
            adaptive_block_parallel = nn.DataParallel( self.adaptive, device_ids=device_ids )

            scores, atten_weights, beta = adaptive_block_parallel( x, hiddens, cells, V )
        else:
            scores, atten_weights, beta = self.adaptive( x, hiddens, cells, V )

        # Return states for Caption Sampling purpose
        return scores, states, atten_weights, beta



# Whole Architecture with Image Encoder and Caption decoder
class Encoder2Decoder( nn.Module ):
    def __init__( self, embed_size, vocab_size, hidden_size ):
        super( Encoder2Decoder, self ).__init__()

        # Image CNN encoder and Adaptive Attention Decoder
        self.encoder = AttentiveCNN( embed_size, hidden_size )
        self.decoder = Decoder( embed_size, vocab_size, hidden_size )


    def forward( self, images, captions, lengths ):

        # Data parallelism for V v_g encoder if multiple GPUs are available
        # V=[ v_1, ..., v_k ], v_g in the original paper
        if torch.cuda.device_count() > 1:
            device_ids = range( torch.cuda.device_count() )
            encoder_parallel = torch.nn.DataParallel( self.encoder, device_ids=device_ids )
            V, v_g = encoder_parallel( images )
        else:
            V, v_g = self.encoder( images )

        # Language Modeling on word prediction
        scores, _, _,_ = self.decoder( V, v_g, captions )

        # Pack it to make criterion calculation more efficient
        packed_scores = pack_padded_sequence( scores, lengths, batch_first=True )

        return packed_scores

    # Caption generator
    def sampler( self, images, max_len=20 ):
        """
        Samples captions for given image features (Greedy search).
        """

        # Data parallelism if multiple GPUs
        if torch.cuda.device_count() > 1:
            device_ids = range( torch.cuda.device_count() )
            encoder_parallel = torch.nn.DataParallel( self.encoder, device_ids=device_ids )
            V, v_g = encoder_parallel( images )
        else:
            V, v_g = self.encoder( images )

        # Build the starting token Variable <start> (index 1): B x 1
        if torch.cuda.is_available():
            captions = Variable( torch.LongTensor( images.size( 0 ), 1 ).fill_( 1 ).cuda() )
        else:
            captions = Variable( torch.LongTensor( images.size( 0 ), 1 ).fill_( 1 ) )

        # Get generated caption idx list, attention weights and sentinel score
        sampled_ids = []
        attention = []
        Beta = []

        # Initial hidden states
        states = None

        for i in range( max_len ):

            scores, states, atten_weights, beta = self.decoder( V, v_g, captions, states )
            predicted = scores.max( 2 )[ 1 ] # argmax
            captions = predicted

            # Save sampled word, attention map and sentinel at each timestep
            sampled_ids.append( captions )
            attention.append( atten_weights )
            Beta.append( beta )

        # caption: B x max_len
        # attention: B x max_len x 49
        # sentinel: B x max_len
        sampled_ids = torch.cat( sampled_ids, dim=1 )
        attention = torch.cat( attention, dim=1 )
        Beta = torch.cat( Beta, dim=1 )


        return sampled_ids, attention, Beta


## UTILS

In [42]:
# Variable wrapper
def to_var(x, volatile=False):
    '''
    Wrapper torch tensor into Variable
    '''
    if torch.cuda.is_available():
        x = x.cuda()
    return Variable( x, volatile=volatile )

# # Show multiple images and caption words
# def show_images(images, cols = 1, titles = None):
#     """Display a list of images in a single figure with matplotlib.
    
#     Parameters
#     ---------
#     images: List of np.arrays compatible with plt.imshow.
    
#     cols (Default = 1): Number of columns in figure (number of rows is 
#                         set to np.ceil(n_images/float(cols))).
    
#     titles: List of titles corresponding to each image. Must have
#             the same length as titles.
            
#     Adapted from https://gist.github.com/soply/f3eec2e79c165e39c9d540e916142ae1
#     """
    
#     assert(( titles is None ) or (len( images ) == len( titles )))
    
#     n_images = len( images )
#     if titles is None: 
#         titles = ['Image (%d)' % i for i in range(1,n_images + 1)]
        
#     fig = plt.figure( figsize=( 15, 15 ) )
#     for n, (image, title) in enumerate( zip(images, titles) ):
        
#         a = fig.add_subplot( np.ceil( n_images/ float( cols ) ), cols, n+1 )
#         if image.ndim == 2:
#             plt.gray()
            
#         plt.imshow( image )
#         a.axis('off')
#         a.set_title( title, fontsize=200 )
        
#     fig.set_size_inches( np.array( fig.get_size_inches() ) * n_images )
    
#     plt.tight_layout( pad=0.4, w_pad=0.5, h_pad=1.0 )
#     plt.show()

# MS COCO evaluation data loader
class CocoEvalLoader( datasets.ImageFolder ):

    def __init__( self, root, ann_path, transform=None, target_transform=None, 
                 loader=datasets.folder.default_loader ):
        '''
        Customized COCO loader to get Image ids and Image Filenames
        root: path for images
        ann_path: path for the annotation file (e.g., caption_val2014.json)
        '''
        self.root = root
        self.transform = transform
        self.target_transform = target_transform
        self.loader = loader
        self.imgs = json.load( open( ann_path, 'r' ) )['images']


    def __getitem__(self, index):

        filename = self.imgs[ index ]['file_name']
        img_id = self.imgs[ index ]['id']
        
        # Filename for the image
        if 'val' in filename.lower():
            path = os.path.join( self.root, 'val2014' , filename )
        else:
            path = os.path.join( self.root, 'train2014', filename )

        img = self.loader( path )
        if self.transform is not None:
            img = self.transform( img )

        return img, img_id, filename

# MSCOCO Evaluation function
def coco_eval( model, epoch, crop_size, image_dir, caption_val, eval_size, num_workers):
    
    '''
    model: trained model to be evaluated
    args: pre-set parameters
    epoch: epoch #, for disp purpose
    '''
    
    model.eval()
    
    # Validation images are required to be resized to 224x224 already
    transform = transforms.Compose([ 
        transforms.Scale( (crop_size, crop_size) ),
        transforms.ToTensor(), 
        transforms.Normalize((0.485, 0.456, 0.406), 
                             (0.229, 0.224, 0.225))])
    
    # Wrapper the COCO VAL dataset
    eval_data_loader = torch.utils.data.DataLoader( 
        CocoEvalLoader(image_dir, caption_val, transform ), 
        batch_size = eval_size, 
        shuffle = False, num_workers = num_workers,
        drop_last = False )  
    
    # Generated captions to be compared with GT
    results = []
    print('---------------------Start evaluation on MS-COCO dataset-----------------------')
    for i, (images, image_ids, _ ) in enumerate( eval_data_loader ):
        
        images = to_var( images )
        generated_captions, _, _ = model.sampler( images )
        
        if torch.cuda.is_available():
            captions = generated_captions.cpu().data.numpy()
        else:
            captions = generated_captions.data.numpy()
        
        # Build caption based on Vocabulary and the '<end>' token
        for image_idx in range( captions.shape[0] ):
            
            sampled_ids = captions[ image_idx ]
            sampled_caption = []
            
            for word_id in sampled_ids:
                
                word = vocab.idx2word[ word_id ]
                if word == '<end>':
                    break
                else:
                    sampled_caption.append( word )
            
            sentence = ' '.join( sampled_caption )
            
            temp = { 'image_id': int( image_ids[ image_idx ] ), 'caption': sentence }
            results.append( temp )
        
        # Disp evaluation process
        if (i+1) % 10 == 0:
            print('[{}{}]'.format( (i+1),len( eval_data_loader ) )) 
            
            
    print('------------------------Caption Generated-------------------------------------')
            
    # Evaluate the results based on the COCO API
    resFile = 'results/mixed-' + str( epoch ) + '.json'
    json.dump( results, open( resFile , 'w' ) )
    
    annFile = caption_val
    coco = COCO( annFile )
    cocoRes = coco.loadRes( resFile )
    
    cocoEval = COCOEvalCap( coco, cocoRes )
    cocoEval.params['image_id'] = cocoRes.getImgIds() 
    cocoEval.evaluate()
    
    # Get CIDEr score for validation evaluation
    cider = 0.
    print('-----------Evaluation performance on MS-COCO validation dataset for Epoch {}----------'.format( epoch ))
    for metric, score in cocoEval.eval.items():
        
        print('{}:{}'.format( metric, score))
        if metric == 'CIDEr':
            cider = score
            
    return cider

## CIDER

In [43]:
import copy
from collections import defaultdict
import numpy as np
import pdb
import math

def precook(s, n=4, out=False):
    """
    Takes a string as input and returns an object that can be given to
    either cook_refs or cook_test. This is optional: cook_refs and cook_test
    can take string arguments as well.
    :param s: string : sentence to be converted into ngrams
    :param n: int    : number of ngrams for which representation is calculated
    :return: term frequency vector for occuring ngrams
    """
    words = s.split()
    counts = defaultdict(int)
    for k in xrange(1,n+1):
        for i in xrange(len(words)-k+1):
            ngram = tuple(words[i:i+k])
            counts[ngram] += 1
    return counts

def cook_refs(refs, n=4): ## lhuang: oracle will call with "average"
    '''Takes a list of reference sentences for a single segment
    and returns an object that encapsulates everything that BLEU
    needs to know about them.
    :param refs: list of string : reference sentences for some image
    :param n: int : number of ngrams for which (ngram) representation is calculated
    :return: result (list of dict)
    '''
    return [precook(ref, n) for ref in refs]

def cook_test(test, n=4):
    '''Takes a test sentence and returns an object that
    encapsulates everything that BLEU needs to know about it.
    :param test: list of string : hypothesis sentence for some image
    :param n: int : number of ngrams for which (ngram) representation is calculated
    :return: result (dict)
    '''
    return precook(test, n, True)

class CiderScorer(object):
    """CIDEr scorer.
    """

    def copy(self):
        ''' copy the refs.'''
        new = CiderScorer(n=self.n)
        new.ctest = copy.copy(self.ctest)
        new.crefs = copy.copy(self.crefs)
        return new

    def __init__(self, test=None, refs=None, n=4, sigma=6.0):
        ''' singular instance '''
        self.n = n
        self.sigma = sigma
        self.crefs = []
        self.ctest = []
        self.document_frequency = defaultdict(float)
        self.cook_append(test, refs)
        self.ref_len = None

    def cook_append(self, test, refs):
        '''called by constructor and __iadd__ to avoid creating new instances.'''

        if refs is not None:
            self.crefs.append(cook_refs(refs))
            if test is not None:
                self.ctest.append(cook_test(test)) ## N.B.: -1
            else:
                self.ctest.append(None) # lens of crefs and ctest have to match

    def size(self):
        assert len(self.crefs) == len(self.ctest), "refs/test mismatch!{}<>{}".format(len(self.crefs), len(self.ctest))
        return len(self.crefs)

    def __iadd__(self, other):
        '''add an instance (e.g., from another sentence).'''

        if type(other) is tuple:
            ## avoid creating new CiderScorer instances
            self.cook_append(other[0], other[1])
        else:
            self.ctest.extend(other.ctest)
            self.crefs.extend(other.crefs)

        return self
    def compute_doc_freq(self):
        '''
        Compute term frequency for reference data.
        This will be used to compute idf (inverse document frequency later)
        The term frequency is stored in the object
        :return: None
        '''
        for refs in self.crefs:
            # refs, k ref captions of one image
            for ngram in set([ngram for ref in refs for (ngram,count) in ref.iteritems()]):
                self.document_frequency[ngram] += 1
            # maxcounts[ngram] = max(maxcounts.get(ngram,0), count)

    def compute_cider(self):
        def counts2vec(cnts):
            """
            Function maps counts of ngram to vector of tfidf weights.
            The function returns vec, an array of dictionary that store mapping of n-gram and tf-idf weights.
            The n-th entry of array denotes length of n-grams.
            :param cnts:
            :return: vec (array of dict), norm (array of float), length (int)
            """
            vec = [defaultdict(float) for _ in range(self.n)]
            length = 0
            norm = [0.0 for _ in range(self.n)]
            for (ngram,term_freq) in cnts.iteritems():
                # give word count 1 if it doesn't appear in reference corpus
                df = np.log(max(1.0, self.document_frequency[ngram]))
                # ngram index
                n = len(ngram)-1
                # tf (term_freq) * idf (precomputed idf) for n-grams
                vec[n][ngram] = float(term_freq)*(self.ref_len - df)
                # compute norm for the vector.  the norm will be used for computing similarity
                norm[n] += pow(vec[n][ngram], 2)

                if n == 1:
                    length += term_freq
            norm = [np.sqrt(n) for n in norm]
            return vec, norm, length

        def sim(vec_hyp, vec_ref, norm_hyp, norm_ref, length_hyp, length_ref):
            '''
            Compute the cosine similarity of two vectors.
            :param vec_hyp: array of dictionary for vector corresponding to hypothesis
            :param vec_ref: array of dictionary for vector corresponding to reference
            :param norm_hyp: array of float for vector corresponding to hypothesis
            :param norm_ref: array of float for vector corresponding to reference
            :param length_hyp: int containing length of hypothesis
            :param length_ref: int containing length of reference
            :return: array of score for each n-grams cosine similarity
            '''
            delta = float(length_hyp - length_ref)
            # measure consine similarity
            val = np.array([0.0 for _ in range(self.n)])
            for n in range(self.n):
                # ngram
                for (ngram,count) in vec_hyp[n].iteritems():
                    # vrama91 : added clipping
                    val[n] += min(vec_hyp[n][ngram], vec_ref[n][ngram]) * vec_ref[n][ngram]

                if (norm_hyp[n] != 0) and (norm_ref[n] != 0):
                    val[n] /= (norm_hyp[n]*norm_ref[n])

                assert(not math.isnan(val[n]))
                # vrama91: added a length based gaussian penalty
                val[n] *= np.e**(-(delta**2)/(2*self.sigma**2))
            return val

        # compute log reference length
        self.ref_len = np.log(float(len(self.crefs)))

        scores = []
        for test, refs in zip(self.ctest, self.crefs):
            # compute vector for test captions
            vec, norm, length = counts2vec(test)
            # compute vector for ref captions
            score = np.array([0.0 for _ in range(self.n)])
            for ref in refs:
                vec_ref, norm_ref, length_ref = counts2vec(ref)
                score += sim(vec, vec_ref, norm, norm_ref, length, length_ref)
            # change by vrama91 - mean of ngram scores, instead of sum
            score_avg = np.mean(score)
            # divide by number of references
            score_avg /= len(refs)
            # multiply score by 10
            score_avg *= 10.0
            # append score of an image to the score list
            scores.append(score_avg)
        return scores

    def compute_score(self, option=None, verbose=0):
        # compute idf
        self.compute_doc_freq()
        # assert to check document frequency
        assert(len(self.ctest) >= max(self.document_frequency.values()))
        # compute cider score
        score = self.compute_cider()
        # debug
        # print score
        return np.mean(np.array(score)), np.array(score)

import pdb

class Cider:
    """
    Main Class to compute the CIDEr metric 
    """
    def __init__(self, test=None, refs=None, n=4, sigma=6.0):
        # set cider to sum over 1 to 4-grams
        self._n = n
        # set the standard deviation parameter for gaussian penalty
        self._sigma = sigma

    def compute_score(self, gts, res):
        """
        Main function to compute CIDEr score
        :param  hypo_for_image (dict) : dictionary with key <image> and value <tokenized hypothesis / candidate sentence>
                ref_for_image (dict)  : dictionary with key <image> and value <tokenized reference sentence>
        :return: cider (float) : computed CIDEr score for the corpus 
        """

        assert(gts.keys() == res.keys())
        imgIds = gts.keys()

        cider_scorer = CiderScorer(n=self._n, sigma=self._sigma)

        for id in imgIds:
            hypo = res[id]
            ref = gts[id]

            # Sanity check.
            assert(type(hypo) is list)
            assert(len(hypo) == 1)
            assert(type(ref) is list)
            assert(len(ref) > 0)

            cider_scorer += (hypo[0], ref)

        (score, scores) = cider_scorer.compute_score()

        return score, scores

    def method(self):
        return "CIDEr"

## TRAINING AND EVALUATION

In [44]:
 # To reproduce training results
torch.manual_seed( 123 )
if torch.cuda.is_available():
    torch.cuda.manual_seed(123)   
# Create model directory
if not os.path.exists( 'C:/Users/aniketdl/Downloads/models' ):
    os.makedirs('C:/Users/aniketdl/Downloads/models')

#### Image preprocessing

In [ ]:
transform = transforms.Compose([ 
    transforms.RandomCrop(224),
    transforms.Resize((256, 256)),
    transforms.RandomHorizontalFlip(), 
    transforms.ToTensor(), 
    transforms.Normalize(( 0.485, 0.456, 0.406 ), 
                             ( 0.229, 0.224, 0.225 ))])

#### Creating Dataloader

In [45]:
# Build training data loader
data_loader = get_loader('C:/Users/aniketdl/Downloads/data', split_data, vocab, 
                              transform, 60,
                              shuffle=True, num_workers=4 ) 

loading annotations into memory...
0:00:00
creating index...
index created!


c:\Miniconda\envs\py38_pytorch\lib\site-packages\torch\utils\data\dataloader.py:478: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2 (`cpuset` is not taken into account), which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


#### Encoder2Decoder Model

In [47]:
# Load pretrained model or build from scratch
adaptive = Encoder2Decoder( 256, len(vocab), 512)
    
start_epoch = 1
    
# Constructing CNN parameters for optimization, only fine-tuning higher layers
cnn_subs = list( adaptive.encoder.resnet_conv.children() )[5: ]
cnn_params = [ list( sub_module.parameters() ) for sub_module in cnn_subs ]
cnn_params = [ item for sublist in cnn_params for item in sublist ]
    
cnn_optimizer = torch.optim.Adam( cnn_params, lr=1e-4, 
                                      betas=(0.8, 0.999) )
    
# Other parameter optimization
params = list( adaptive.encoder.affine_a.parameters() ) + list( adaptive.encoder.affine_b.parameters() ) \
                + list( adaptive.decoder.parameters() )

# Will decay later    
learning_rate = 4e-4
    
# Language Modeling Loss
LMcriterion = nn.CrossEntropyLoss()
    
# Change to GPU mode if available
if torch.cuda.is_available():
        adaptive.cuda()
        LMcriterion.cuda()
    
# Train the Models
total_step = len( data_loader )
    
cider_scores = []
best_cider = 0.0
best_epoch = 0

Downloading: "https://download.pytorch.org/models/resnet152-394f9c45.pth" to C:\Users\aniketdl/.cache\torch\hub\checkpoints\resnet152-394f9c45.pth


<ipython-input-41-aa3374846ec6>:25: UserWarning: nn.init.kaiming_uniform is now deprecated in favor of nn.init.kaiming_uniform_.
  init.kaiming_uniform(self.affine_a.weight, mode='fan_in')  # He weight initialization
<ipython-input-41-aa3374846ec6>:26: UserWarning: nn.init.kaiming_uniform is now deprecated in favor of nn.init.kaiming_uniform_.
  init.kaiming_uniform(self.affine_b.weight, mode='fan_in')
<ipython-input-41-aa3374846ec6>:121: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
  init.xavier_uniform(self.affine_x.weight)
<ipython-input-41-aa3374846ec6>:122: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
  init.xavier_uniform(self.affine_h.weight)
<ipython-input-41-aa3374846ec6>:68: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
  init.xavier_uniform(self.affine_v.weight)
<ipython-input-41-aa3374846ec6>:69: UserWarning: nn.init.xavier_uniform is now 

#### TRAINING

In [ ]:
# Start Training 
for epoch in range( start_epoch, 2 ):

    # Start Learning Rate Decay
    if epoch > 20:
                
            frac = float( epoch - 20) / 50
            decay_factor = math.pow( 0.5, frac )

            # Decay the learning rate
            learning_rate = 4e-4 * decay_factor
    print('Learning Rate for Epoch {}: {}'.format( epoch, learning_rate))

    optimizer = torch.optim.Adam( params, lr=learning_rate, betas=(0.8, 0.999) )

    # Language Modeling Training
    print('------------------Training for Epoch {}----------------'.format(epoch))
    for i, (images, captions, lengths, _, _ ) in enumerate( data_loader ):

            # Set mini-batch dataset
            images = to_var( images )
            captions = to_var( captions )
            lengths = [ cap_len - 1  for cap_len in lengths ]
            targets = pack_padded_sequence( captions[:,1:], lengths, batch_first=True )[0]

            # Forward, Backward and Optimize
            adaptive.train()
            adaptive.zero_grad()

            packed_scores = adaptive( images, captions, lengths )

            # Compute loss and backprop
            loss = LMcriterion( packed_scores[0], targets )
            loss.backward()
            
            # Gradient clipping for gradient exploding problem in LSTM
            for p in adaptive.decoder.LSTM.parameters():
                p.data.clamp_(-0.1, 0.1)
            
            optimizer.step()
            
            # Start CNN fine-tuning
            if epoch > 20:

                cnn_optimizer.step()

            # Print log info
            if i % args.log_step == 0:
                print('Epoch [{}{}], Step [{}{}], CrossEntropy Loss: {}, Perplexity: {}'.format( epoch, 
                                                                                                 1, 
                                                                                                 i, total_step, 
                                                                                                 loss.data[0],
                                                                                                 np.exp( loss.data[0] ) ))  
                
    # Save the Adaptive Attention model after each epoch
    torch.save( adaptive.state_dict(), 
                    os.path.join('C:/Users/aniketdl/Downloads/models', 
                    'adaptive-%d.pkl'%( epoch ) ) )          
      
        
    # Evaluation on validation set        
    cider = coco_eval( adaptive, epoch, crop_size=224, image_dir='C:/Users/aniketdl/Downloads/data', caption_val=split_data, eval_size=28, 
                      num_workers=4)
    cider_scores.append( cider )        
        
    if cider > best_cider:
            best_cider = cider
            best_epoch = epoch
       
    if len( cider_scores ) > 5:
            
            last_6 = cider_scores[-6:]
            last_6_max = max( last_6 )
            
            # Test if there is improvement, if not do early stopping
            if last_6_max != best_cider:
                
                print('No improvement with CIDEr in the last 6 epochs...Early stopping triggered.')
                print('Model of best epoch #: {} with CIDEr score {}'.format( best_epoch, best_cider))
                break


Learning Rate for Epoch 1: 0.0004
------------------Training for Epoch 1----------------
